In [1]:
# Laura Chapman
# Script for developing statistical tools for Glue
# Computes statistics for subsets as well as entire data using compute_statistic
# Changed from astropy tables to pandas dataframe

# Format data well in a popup using qt
# Color code by subset

In [2]:
# Basic code that imports glue and loads in and links the data

import sys
from glue.core.data_factories import load_data
from glue.core import DataCollection
from glue.core.link_helpers import LinkSame
from glue.app.qt.application import GlueApplication
from glue.viewers.image.qt import ImageViewer
from glue_vispy_viewers.volume.volume_viewer import VispyVolumeViewer

image_filename='w5.fits'
catalog_filename='w5_psc.vot'

#load 2 datasets from files
catalog = load_data(catalog_filename)
image = load_data(image_filename)

dc = DataCollection([catalog,image])

# link positional information
dc.add_link(LinkSame(catalog.id['RAJ2000'], image.id['Right Ascension']))
dc.add_link(LinkSame(catalog.id['DEJ2000'], image.id['Declination']))

#Create subset based on filament mask
ra_state=(image.id['Right Ascension'] > 44) & (image.id['Right Ascension'] < 46)
subset_group=dc.new_subset_group('RA_Selection',ra_state)
subset_group.style.color = '#0000FF'



In [3]:
import glue.utils.array as gua
import glue.core.data as gcd
from astropy.table import Table

In [4]:
# Constructs a pandas DataFrame

import pandas as pd
from pandas import DataFrame
import numpy as np

In [5]:
# Defines a subset and runs statistics using compute_statistic for the subset
# Saves data using a pandas dataframe called my_pandas_data

# Data that subset is pulled from
data = dc[0]

# Define state and subset
state1 = data.id['Jmag'] > 14
subset1 = data.new_subset(state1, label='Jmag > 14')

# Same arrays as with full data
mean_array = []
median_array = []
min_array = []
max_array = []
sum_array = []
name_array = []
tables = []

headings = ('mean', 'median', 'minimum', 'maximum', 'sum')

print(data.label, 'subset1')
print() 
for j in range (0, len(data.components)):
    name = data.components[j].label # Get the name of each component
    name_array.append(name) # add to the name array to build the table
    mean_array.append(data.compute_statistic('mean', subset1.components[j], subset_state=subset1.subset_state))
    median_array.append(data.compute_statistic('median', subset1.components[j], subset_state=subset1.subset_state))       
    min_array.append(data.compute_statistic('minimum', subset1.components[j], subset_state=subset1.subset_state))       
    max_array.append(data.compute_statistic('maximum', subset1.components[j], subset_state=subset1.subset_state))      
    sum_array.append(data.compute_statistic('sum', subset1.components[j], subset_state=subset1.subset_state))        
   
column_data = np.asarray([mean_array, median_array, min_array, max_array, sum_array]).transpose()

my_pandas_data = pd.DataFrame(column_data, index=name_array, columns=headings)

w5_psc subset1



In [ ]:
from PyQt5.QtWidgets import QApplication, QComboBox, QWidget, QPushButton, QHBoxLayout, QTableView,QGroupBox, QDialog, QVBoxLayout, QLabel,QGridLayout
from PyQt5.QtGui import QIcon
from PyQt5.QtCore import pyqtSlot, QVariant
from PyQt5 import QtCore
import sys

from PyQt5 import QtCore, QtWidgets, QtGui
Qt = QtCore.Qt
# from PyQt5.QtCore import QVariant
from qtpy.QtWidgets import (QApplication, QLabel, QWidget, QComboBox, QCheckBox,
                            QVBoxLayout, QPushButton, QGridLayout, QTableView) 

class StatsGui(QWidget):
    ''' 
    This class accepts a glue data collection object, and builds an interactive window
    to display basic statistics (e.g. mean, median, mode) about each dataset
    '''
    def __init__(self,dc):
        
        # Initialize the object as a QWidget
        QWidget.__init__(self)
    
        #Save the datacollection object as an attribute of class StatsGui
        self.dc=dc

        #Create a dictionary which will keep track of which QPushButtons are selected for each dataset
        self.toggle_dictionary = dict((dc.labels[i], np.zeros(len(dc[i].components))) for i in range(0, len(dc)))      
        
        #Fix the size of the main GUI window (for now)
        self.setMinimumSize(800, 600)
        self.setMaximumSize(800, 600)
        
        #Set the title of the main GUI window
        self.setWindowTitle('Statistics')
        
        #################Data Selection Widget#############################
        #Set up the viewer that will contain the dropbown data selection menu
        self.choose_data = QComboBox(self)
        self.choose_data.addItems(["Choose a dataset"])
        self.choose_data.addItems(dc.labels)
        
        #Disable the "Choose a dataset" button, and move the dropdown menu to center of GUI
        self.choose_data.model().item(0).setEnabled(False)
        self.choose_data.move(300, 5)
        
        #When a dataset is changed, call "selectionchange" function to update QGridBox widget
        self.choose_data.currentIndexChanged.connect(self.selectionchange)    
            
        #################Set up the QTableView Widget#############################
        self.table = QTableView(self)
        # Set up the table with the stat headings
        self.headings = ('mean', 'median', 'minimum', 'maximum', 'sum')   
        self.data_frame = pd.DataFrame(columns=self.headings)                      
        self.model = pandasModel(self.data_frame)

        self.table.setModel(self.model)
        self.table.setShowGrid(False)          
        
        #Move the table widget to the bottom of the GUI window
        self.table.setGeometry(50, 250, 700, 300) 

        #################Set up the QGroupBox Widget#############################
        #This widget creates a frame to hold QGridLayout, which will itself hold array of QPushButtons
        self.horizontalGroupBox = QGroupBox(self)
        
        #Move QGroupBox to middle of GUI window
        self.horizontalGroupBox.setGeometry(50, 50, 700, 200) 

        
    def clicked(self):
        '''
        This function is activated whenever a component button is pressed inside the QGridLayout.
        Each time it is called, it will determine which QPushButton was pressed or unpressed.
        It will keep track of toggle state of all buttons for all datasets inside the toggle_dictionary variable
        WARNING: For now, this function only works if component names within a dataset are unique
        '''
        
        button=self.sender()
        
        #Check to make sure the button is indeed a QPushButton
        if isinstance(button, QPushButton):
            
            #If the button is clicked, figure out which one, and log the "on" state in toggle_dictionary
            #If the button is clicked, calculate its statistics and add a row to the table
            if button.isChecked():
                comp_i = np.where(np.asarray(dc[self.data_index].components).astype(str) == button.text())[0][0]                
                self.toggle_dictionary[dc.labels[self.data_index]][comp_i] = 1
                
                # Run statistics
                
                # Set up the statistical arrays and clear each
                name_array = []
                mean_array = []
                median_array = []
                min_array = []
                max_array = []
                sum_array = []
                  
                # Retrive the statistical data and add it to the arrays
                name = self.dc[self.data_index].components[comp_i].label
                name_array.append(name) # add to the name array to build the table
                mean_array.append(self.dc[self.data_index].compute_statistic('mean', self.dc[self.data_index].components[comp_i]))
                median_array.append(self.dc[self.data_index].compute_statistic('median', self.dc[self.data_index].components[comp_i]))       
                min_array.append(self.dc[self.data_index].compute_statistic('minimum', self.dc[self.data_index].components[comp_i]))      
                max_array.append(self.dc[self.data_index].compute_statistic('maximum', self.dc[self.data_index].components[comp_i]))     
                sum_array.append(self.dc[self.data_index].compute_statistic('sum', self.dc[self.data_index].components[comp_i]))             
                       
                column_data = np.asarray([mean_array, median_array, min_array, max_array, sum_array]).transpose()                
            
                self.data_frame = pd.DataFrame(column_data, index=name_array, columns=self.headings)
                
# this code attempted to add and remove rows into the pandas model- not functional yet             
#                 try: 
#                     self.model
#                     self.model.insertRow(0)
                    
#                 except:
#                     self.model = pandasModel(self.data_frame)                    
#                     self.table.setModel(self.model)

                self.model = pandasModel(self.data_frame)                    
                self.table.setModel(self.model)

            #If a button is clicked off, figure out which one, and log the "off" state in toggle_dictionary
            #I the button is clicked off, remove that row from the table
            else:
                index = np.where(np.asarray(dc[self.data_index].components).astype(str) == button.text())[0]
                self.toggle_dictionary[dc.labels[self.data_index]][index] = 0
                # Attempt to remove the row
#                 self.model.removeRow(0)
                
    def selectionchange(self, i):
        '''
        This function is called whenever the user selects a new dataset inside QComboBox
        Its job is to replace the QPushButtons in QGridLayout with the components corresponding to the dataset.
        If a QGridLayout already exists, it removes all the old QPushButtons before adding new ones
        '''        
        
        #"Choose a dataset" counts as a line in QComboBox, so we must subtract one to start from index zero
        self.data_index = i-1
    
        #Check to see if a QGridLayout already exists. 
        try:
            self.component_layout
            
            #While there are QPushButtons in the QGridLayout widget, delete them, starting from the first button on grid
            while self.component_layout.count():
                item = self.component_layout.takeAt(0)
                widget = item.widget()
                widget.deleteLater()
                
        #If QGridLayout doesn't exist, make a new one and call it component_layout
        except:
            self.component_layout = QGridLayout()
            
            
        #Now set up all the buttons; We want three columns and the minimum number of rows needed to hold all component buttons
        ncols=3
        nrows=int(np.round((len(self.dc[self.data_index].components)/3.)+0.5))

        #Loop through nrows and ncols to create the grid of QPushButtons so users can select data components
        i=0
        for col in range(0,nrows):
            for row in range(0,ncols):
                
                #Set the text on the button equal to the component name
                btnlabel=str(self.dc[self.data_index].components[i])
                
                #Create the button with that text
                btn=QPushButton(btnlabel)
                
                #Make sure the button can be checked
                btn.setCheckable(True)
                
                #Allow the button to be toggled on AND off
                btn.toggle()
                
                #Whenever a button is clicked, pass it to the "clicked" function
                btn.clicked.connect(self.clicked)
                
                #Add the button to the QGridLayout widget
                self.component_layout.addWidget(btn,col,row)
                
                #Set the button to be on or off based on the toggle state saved in toggle_dictionary
                if self.toggle_dictionary[dc.labels[self.data_index]][i] == 0:
                    btn.setChecked(False)
                else:
                    btn.setChecked(True)
                    
                i=i+1
            
                #Do not add more buttons than there are components in data
                if i==len(dc[self.data_index].components):
                    break    
                
        #Now add the QGridLayout with all the QPushButtons to the QGroupBoxs
        self.horizontalGroupBox.setLayout(self.component_layout)
    
class pandasModel(QtCore.QAbstractTableModel):
    # Set up the data in a form that allows it to be added to qt widget
    # Write a function that allows data to be updated ie add and remove rows
    def __init__(self, df, parent=None):
        QtCore.QAbstractTableModel.__init__(self, parent)
        self.data_frame = df
        super(pandasModel, self).__init__(parent)      

    def rowCount(self, parent=None):
        return len(self.data_frame.values)

    def columnCount(self, parent=None):
        return self.data_frame.columns.size

    def data(self, index, role=Qt.DisplayRole):
        if index.isValid():
            if role == Qt.DisplayRole:
                return QVariant(str(
                    self.data_frame.values[index.row()][index.column()]))
        return QVariant()
    
    def headerData(self, col, orientation, role):
        if orientation == Qt.Horizontal and role == Qt.DisplayRole:
            return QVariant(self.data_frame.columns[col])
        if orientation == Qt.Vertical and role == Qt.DisplayRole:
            return QVariant(self.data_frame.index[col])
        return QVariant()  
    
 # Trying to add functions that allow for rows to be removed and inserted into the pandas model
#     def removeRow(self, position, rows=1, index=QModelIndex()):
#         self.beginRemoveRows(QModelIndex(), position, position + rows - 1)       
#         self.data_frame = self.data_frame[:position] + self.items[position + rows:]
#         self.endRemoveRows()

#         return True

#     def insertRows(self, position, rows=1, index=QModelIndex()):
#         self.beginInsertRows(QModelIndex(), position, position + rows - 1)
#         for row in range(rows):
#             self.data_frame.insert(position, 0, [2,3,4,5,6])
#             self.added+=1
#         self.endInsertRows()
#         return True   
    
app = QApplication.instance()
if app is None:
    app = QApplication(sys.argv)
else:
    print('QApplication instance already exists: %s' % str(app))
ex = StatsGui(dc)
ex.show()
sys.exit(app.exec_())

In [ ]:
'''Things to implement
add the name of the data set to the component
sort by maximum, minimum etc
make it scroll if there are too many buttons
have a drop down menu to select components that you want to turn into buttons
have a button that says select all or deselect all
show subsets as a button
have a list of components and select from a list instead of buttons
complete working insert row and remove row functionality when user clicks buttons
add in data subsets
color code table by data/subset?
export to file button
advanced mode that allows user to pick what stats to calculate'''